<a href="https://colab.research.google.com/github/odesay97/MachineLearing_Class/blob/main/5%EC%9E%A5_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 데이터 불러오기 및 전처리

import pandas as pd

dataset = pd.read_csv("https://bit.ly/wine_csv_data")

# dataset.head()
# dataset.info()
# dataset.describe()

import numpy as np

train_data = dataset[['alcohol','sugar','pH']]
target_data = dataset['class']

from sklearn.model_selection import train_test_split, GridSearchCV

train_input, test_input, train_target, test_target = train_test_split(train_data,target_data,stratify=target_data,random_state=46,test_size=0.2)

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)






결정트리

In [4]:
# 결정트리 최적화
from sklearn.tree import DecisionTreeClassifier

params = { 'max_depth': np.arange(3,20,1),
          'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'min_samples_split': np.arange(2,40,3),
}

gs1 = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=1)

gs1.fit(train_input,train_target)

dt = gs1.best_estimator_

print(gs1.best_params_)
print(dt.score(train_input,train_target))
print(dt.score(test_input,test_target))


{'max_depth': 13, 'min_impurity_decrease': 0.00030000000000000003, 'min_samples_split': 8}
0.9084564860426929
0.8738461538461538


In [ ]:
# 가장 높은 중요성을 가진 sugar특성만을 가지고, 특성 공학을 사용하여 나타내보기

from sklearn.model_selection import train_test_split

import numpy as np

train_data_A = dataset[["sugar"]].to_numpy()
target_data_A = dataset["class"].to_numpy()

from sklearn.model_selection import train_test_split

train_input_A, test_input_A, train_target_A, test_target_A = train_test_split(train_data_A,target_data_A,random_state=55)


from sklearn.preprocessing import PolynomialFeatures, StandardScaler


pf = PolynomialFeatures(degree=3)
pf.fit(train_input_A)
train_poly = pf.transform(train_input_A)
test_poly = pf.transform(test_input_A)

ss = StandardScaler()
ss.fit(train_poly)

train_poly_scaled = ss.transform(train_poly)
test_poly_scaled = ss.transform(test_poly)

############
from sklearn.tree import DecisionTreeClassifier

params = { 'max_depth': np.arange(3,20,1),
          'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'min_samples_split': np.arange(2,40,3),
}

gs1 = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=1)
gs1.fit(train_poly_scaled,train_target_A)

print(gs1.score(train_poly_scaled,train_target_A))
print(gs1.score(test_poly_scaled,test_target_A))
print(gs1.best_params_)

# 위의 방식보다 격차는 줄었지만 성능이 떨어졌음 -> 좋지 않은 시도로 보임

0.8399014778325123
0.8283076923076923
{'max_depth': 5, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}


In [5]:
# 엔트로피 불순도를 사용하여 최적화
from sklearn.tree import DecisionTreeClassifier

params = { 'max_depth': np.arange(3,20,1),
          'min_impurity_decrease': np.arange(0.0001,0.001,0.0001),
          'min_samples_split': np.arange(2,40,3)
}

gs1 = GridSearchCV(DecisionTreeClassifier(random_state=42, criterion='entropy'),params,n_jobs=1)
gs1.fit(train_input,train_target)

dt = gs1.best_estimator_

print(dt.score(train_input,train_target))
print(dt.score(test_input,test_target))
print(gs1.best_params_)

# 과대적합이 조금 더 해결됨

0.8840311986863711
0.8707692307692307
{'max_depth': 9, 'min_impurity_decrease': 0.0009000000000000001, 'min_samples_split': 2}


랜덤 포레스트

In [9]:
# 랜덤 포레스트

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1,random_state=49,oob_score=True)
scores = cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)

import numpy as np
print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 과대적합 상태


{'fit_time': array([1.4081924 , 1.30694771, 1.2198329 , 1.11660886, 0.85328317]), 'score_time': array([0.12243986, 0.10814619, 0.07639885, 0.10161257, 0.0811007 ]), 'test_score': array([0.87788462, 0.9       , 0.88354187, 0.88065448, 0.89316651]), 'train_score': array([0.99759442, 0.99663219, 0.9968735 , 0.998557  , 0.9978355 ])}
0.9974985197982551 0.8870494928555563


In [14]:
# 그리드 서치를 통해 하이퍼파라미터 튜닝 후 학습

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

params = {
    'max_depth': np.arange(2,11,1),
    'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
    'min_samples_split' : np.arange(2,20,2)
}

gs1 = GridSearchCV(RandomForestClassifier(n_jobs=-1,random_state=42),params,n_jobs=-1)
gs1.fit(train_input,train_target)
rf = gs1.best_estimator_

print(rf.score(train_input,train_target))
print(rf.score(test_input,test_target))
print(gs1.best_params_)

# 과대적합이 많이 개선됨


0.9101404656532615
0.8807692307692307
{'max_depth': 10, 'min_impurity_decrease': 0.0001, 'min_samples_split': 8}


In [15]:
# 그리드 서치를 통해 하이퍼파라미터 튜닝 후 학습 (엔트로피)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

params = {
    'max_depth': np.arange(2,11,1),
    'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
    'min_samples_split' : np.arange(2,20,2)
}

gs1 = GridSearchCV(RandomForestClassifier(n_jobs=-1,random_state=42, criterion='entropy'),params,n_jobs=-1)
gs1.fit(train_input,train_target)
rf = gs1.best_estimator_

print(rf.score(train_input,train_target))
print(rf.score(test_input,test_target))
print(gs1.best_params_)

# 성능의 미묘한 향상이 있음 -> 크게 유의미하진 않음

0.9097556282470656
0.8823076923076923
{'max_depth': 10, 'min_impurity_decrease': 0.0002, 'min_samples_split': 6}


엑스트라 트리

In [16]:
# 엑스트라 트리
from sklearn.model_selection import cross_validate
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1,random_state=49,oob_score=True)
scores = cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)

import numpy as np
print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


ValueError: ignored

그레디언트 부스팅

In [ ]:
# 그레디언트 부스팅

히스토그램기반 그레디언트 부스팅

In [ ]:
# 히스토그램기반 그래디언트 부스팅